In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
#from keras.models import Model, load_model
#from keras.layers import LSTM, Dense, TimeDistributed, Input, Masking, RepeatVector, Bidirectional, Embedding
nltk.download('punkt')
import string
punc = list(string.punctuation)
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.metrics import pairwise
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pickle
import csv
import os
from scipy.spatial.distance import cosine
import time
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.feature_extraction.text import CountVectorizer
import torch

In [3]:
## HYPER-PARAMETERS

num_words = 10000
MAX_SEQUENCE_LENGTH = 16
WORD_EMBEDDING_DIM = 200
batch_size = 640
ENCODING_DIM = 500
NUM_DATAPOINT = -1 # All in the file
NUM_ICD_CODES = 2833
NUM_EPOCHS = 2
VALIDATION_SPLIT = 0.2
PATIENCE = 5
TOP_K_PER_SENTENCE = 10
similarity_threshold = 0.7

THRESHOLD = -1 # TODO

ICD_DESC_FILENAME = 'ICD_desc_with_freq.csv'
#DD_FILENAME = 'MIMIC_III_Final_1000_clean_shorted.csv'
#DD_FILENAME = 'MIMIC-III-Final_5000_clean_shortened.csv'
DD_FILENAME = 'MIMIC-III-Final_cleaned.csv'
EMB_FILE = 'PMC-w2v.bin'

#RESULT_FILENAME = 'Experiment Results.xlsx'
#http://yaronvazana.com/2019/09/28/training-an-autoencoder-to-generate-text-embeddings/

In [4]:
#Calculate Sentence Embedding using Average Word Embedding of tokens
def get_sentence_embedding(sent, embedding):
    words = word_tokenize(sent)
    m = len(words)
    total_embedding = 0
    
    #Get wordwise Embeddings as tensors
    sent_embedding = np.array([get_word_embedding(word, embedding) for word in words])
    
    #Get the embedding of sentence by adding embedding tensors
    total_embedding = np.sum(sent_embedding, axis = 0)
    
    try:
        return total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True)
    except:
        #print('sent : ', sent)
        #print('m: ', m)
        return np.array(total_embedding/np.linalg.norm(total_embedding, axis = 0, keepdims=True))

In [5]:
#Calculate Word Embedding
def get_word_embedding(word, embedding):
    try:
        return embedding[word] 
    except:
        return embedding['unk']

In [6]:
#Load Word Embedding (Pub Med W2V)
from gensim.models.keyedvectors import KeyedVectors
word_model = KeyedVectors.load_word2vec_format(EMB_FILE, binary=True)
print('Found %s word vectors.' % len(word_model.index2word))
print(word_model['Cholera'].shape)

Found 2515686 word vectors.
(200,)


In [7]:
word_model['Cholera']/np.linalg.norm(word_model['Cholera'], 2)

array([-0.00481831,  0.13864845, -0.0831728 , -0.00180067, -0.00696073,
       -0.02875327,  0.06748949,  0.081326  ,  0.11438117,  0.00530638,
       -0.05866855,  0.05307228, -0.09064269,  0.03729252, -0.00409504,
        0.02162931,  0.11918882, -0.05017674,  0.03869108,  0.12593974,
       -0.06599744,  0.13589439,  0.02106961, -0.02205387,  0.04005024,
        0.11348598, -0.09267536, -0.04086379, -0.07475508,  0.02114647,
        0.03469999,  0.02900406,  0.06926779,  0.11697073, -0.01642677,
       -0.00433   ,  0.06071504, -0.1460445 , -0.01645673, -0.0205683 ,
        0.01536769,  0.0160432 , -0.00429431,  0.03412509, -0.05015106,
        0.0433115 ,  0.08293187,  0.14879459, -0.0841924 , -0.06157425,
       -0.02435092, -0.05808348, -0.11018413,  0.01283847, -0.07155055,
        0.08904947, -0.0320092 , -0.11904893, -0.01103841,  0.11339661,
        0.01087772, -0.05354536,  0.0755807 ,  0.07982337,  0.11575826,
       -0.1831625 ,  0.01570357, -0.05355467,  0.065675  , -0.05

# ICD File

In [8]:
ICD_desc_filename = ICD_DESC_FILENAME
df_icd = pd.read_csv(ICD_desc_filename, encoding = 'latin1')

df_icd = df_icd[['Code', 'Long Description']]

print(df_icd.shape)

print('\n', df_icd.head())

(6553, 2)

     Code                                   Long Description
0   4019                 Unspecified essential hypertension
1   4280              Congestive heart failure, unspecified
2  42731                                Atrial fibrillation
3  41401  Coronary atherosclerosis of native coronary ar...
4   5849                  Acute kidney failure, unspecified


In [9]:
df_icd_cleaned = df_icd
df_icd_cleaned['Sentence Embedding'] = ''

for i in range(df_icd.shape[0]):
    sent = df_icd_cleaned['Long Description'][i]
    sent_tokenized = word_tokenize(sent)
    
    clean_sent = ''
    
    for token in sent_tokenized:
        if token.lower() not in punc and token.lower() not in stop_words:
            clean_sent = clean_sent + ' ' + token.lower()
    
    df_icd_cleaned['Long Description'][i] = clean_sent
    
    #df_icd_cleaned['Sentence Embedding'][i] = get_sentence_embedding(df_icd_cleaned['Long Description'][i], word_model)
    
df_icd_cleaned.head()    

,Code,Long Description,Sentence Embedding
0,4019,unspecified essential hypertension,
1,4280,congestive heart failure unspecified,
2,42731,atrial fibrillation,
3,41401,coronary atherosclerosis native coronary artery,
4,5849,acute kidney failure unspecified,


In [10]:
#Capture the ICD embedding as a numpy matrix
#icd_sent_emb = df_icd_cleaned['Sentence Embedding'][:]
#icd_sent_emb_list = [x for x in icd_sent_emb]

#icd_sent_emb_matrix = np.asarray(icd_sent_emb_list, dtype=np.float32)
#icd_sent_emb_matrix.shape

In [11]:
icd_sentences = list(df_icd_cleaned.head(NUM_ICD_CODES)['Long Description']) # Number of sentences
icd_codes = list(df_icd_cleaned.head(NUM_ICD_CODES)['Code'])
icd_index = list(range(NUM_ICD_CODES))

print(len(icd_sentences), len(icd_codes), len(icd_index))
print("ICD_Sentences = ", len(icd_sentences))

2833 2833 2833
ICD_Sentences =  2833


In [13]:
# 3 Dictionary of ICD index, ICD Code and ICD desc

# index, codes
dict_index_code = dict(zip(icd_index, icd_codes))

# codes, index
dict_code_index = dict(zip(icd_codes, icd_index))

#index, code desc
dict_index_desc = dict(zip(icd_index, icd_sentences))

#icd sentence embedding matrix slice
#icd_sent_emb_matrix = icd_sent_emb_matrix[:NUM_ICD_CODES]

# Cleaned MIMIC-III DD file

In [14]:
DD_filename = DD_FILENAME
df_DD = pd.read_csv(DD_filename)
df_DD = df_DD[15000:30000]
df_DD.reset_index(drop=True, inplace= True)
FILESIZE = df_DD.shape[0]
df_DD.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,clean_text,top_ICD
0,15000,{Admission Date: [**2193-3-31**] ...,128349.0,"['42823', '51882', '5849', '3371', '4280', '41...",admission date discharge date known allergies ...,"['42823', '51882', '5849', '3371', '4280', '41..."
1,15001,{Admission Date: [**2152-11-17**] Disch...,128350.0,"['5602', '2765', '5990', '4280', '42731', '244...",admission date discharge date general of prese...,"['5602', '5990', '4280', '42731', '2449', '4019']"
2,15002,{Unit No: [**Numeric Identifier 66906**]\nAdm...,128357.0,"['41011', '41401', '27801', '3051']",unit no numeric identifier date date of birth ...,"['41011', '41401', '27801', '3051']"
3,15003,{Admission Date: [**2108-12-10**] ...,128359.0,"['51884', '5070', '48282', '4821', '2874', '27...",admission date discharge date of birth sex rec...,"['51884', '5070', '48282', '4821', '2763', '99..."
4,15004,{Admission Date: [**2102-12-30**] Dischar...,128360.0,"['41071', '99812', '4280', '41401', '2449']",admission date discharge date of birth sex for...,"['41071', '99812', '4280', '41401', '2449']"


In [15]:
FILESIZE

15000

In [16]:
DD_Data = df_DD[['ICD9_CODE', 'clean_text']]
del df_DD
DD_Data

,ICD9_CODE,clean_text
0,"['42823', '51882', '5849', '3371', '4280', '41...",admission date discharge date known allergies ...
1,"['5602', '2765', '5990', '4280', '42731', '244...",admission date discharge date general of prese...
2,"['41011', '41401', '27801', '3051']",unit no numeric identifier date date of birth ...
3,"['51884', '5070', '48282', '4821', '2874', '27...",admission date discharge date of birth sex rec...
4,"['41071', '99812', '4280', '41401', '2449']",admission date discharge date of birth sex for...
...,...,...
14995,"['51881', '99681', '5849', '4280', '42731', '3...",admission date discharge date of birth sex rec...
14996,"['8730', '5990', '8245', '81383', '78009', '40...",admission date discharge date of birth sex of ...
14997,"['53140', '4821', '51881', '42731', '496', '40...",admission date discharge date of birth sex of ...
14998,"['53240', '99664', '5990', '2851', 'E9359', '5...",admission date discharge date recorded as havi...


In [ ]:
Vocab = {}

doc_len = []

for index in range(DD_Data.shape[0]):
    sent = word_tokenize(DD_Data['clean_text'][index])
    
    doc_len.append(len(sent))
    
    for word in sent:
        if word in Vocab.keys():
            Vocab[word]+=1
        else:
            Vocab[word]=1

print('Vocab Size: ', len(Vocab.keys()))
print('Avg Doc Len: ', float(sum(doc_len))/float(len(doc_len)))

In [ ]:
del Vocab, doc_len

In [ ]:
#Get TF vector using entire corpus
def get_tf(docs):
    
    vectorizer = CountVectorizer()#CountVectorizer(stop_words = 'english')
    X = vectorizer.fit_transform(docs)#(docs['sentence'])

    # place tf values in a pandas data frame
    tf_vecs = pd.DataFrame(X.T.todense(), index = vectorizer.get_feature_names())
    
    #Take sum accross sentences
    freq_vec = tf_vecs.sum(axis = 1)
    freq_vec = pd.DataFrame(freq_vec, index=freq_vec.index, columns=['Frequency'])
    
    return vectorizer, freq_vec

In [ ]:
#remove terms not present in freq_vec
def remove_unwanted_words(words, freq_vec):
    
    pruned_words = [x for x in words if x in freq_vec.index]
    
    #for i in words:
    #    if i in freq_vec.index:
    #        pruned_words.append(i)
    
    return list(set(pruned_words))

In [ ]:
def get_tf_vector(sent, freq_vec):
    #tokenize sentence
    words = list(set(word_tokenize(sent)))
    
    #remove terms not occuring in freq_vec
    pruned_words = remove_unwanted_words(words, freq_vec)
    
    #get the slice of freq_vec using the remaining terms
    #print(words, pruned_words)
    freq_slice = freq_vec.loc[pruned_words, :]
    
    #L1 normalize to obtain tf vector
    tf_vec = freq_slice/np.linalg.norm(freq_slice, ord = 1)    

    return tf_vec, pruned_words

In [ ]:
#calculate the cosine similarity matrix for 2 sentences
def cosine_sim_matrix(s1, s2):
    
    #Get token-wise embedding
    s1_token_emb = np.array([get_word_embedding(word, word_model) for word in s1])
    #Normalize
    s1_token_emb/=np.linalg.norm(s1_token_emb, axis = 1, ord = 2, keepdims=True)
    
    #Get token-wise embedding of icd text
    s2_token_emb = np.array([get_word_embedding(word, word_model) for word in s2])
    #Normalize
    s2_token_emb/=np.linalg.norm(s2_token_emb, axis = 1, ord = 2, keepdims=True)
    
    
    sim_matrix = s1_token_emb.dot(s2_token_emb.T)
    
    return np.clip(sim_matrix, -1.0, 1.0) #Clip the Elements of the matrix

In [32]:
#Reset Debugging Counter
k = 0
#Calculate the Similarity Score between 2 senteces
def RWMS_similarity(sent_1, sent_2, freq_vec):
    global k
    try:
        
        #get tf vectors
        #start = time.time()
        sent1_tf_vec, sent1 = get_tf_vector(sent_1, freq_vec)
        #sent2_tf_vec, sent2 = get_tf_vector(sent_2, freq_vec)
        
        #tokenized sentences
        sent1 = sent1#list(set(word_tokenize(sent_1)))
        sent2 = list(set(word_tokenize(sent_2)))

        #Compute Cosine Similarity Matrix
        sim_mat = cosine_sim_matrix(sent1, sent2)
        #print('Sim_mat:\n', sim_mat)
        
        #Initialise Similarity values:
        W_s_t = 0
        #W_t_s = 0

        #for S->t (icd_text -> dd text)
        max_cosine_vec = sim_mat.max(axis = 1)
        #print('Sim_mat:\n', sim_mat)
        
        # Take Elementwise product of max cos similarity vector and tf vector
        sim_vec = sent1_tf_vec.to_numpy().transpose() * max_cosine_vec
        
        #Take sum of all elements of sim_vec to obtain final similarity
        W_s_t = np.sum(sim_vec)
        
        del sim_mat, sent1_tf_vec, sent1, sent2
        
        #for t->S
        #for j in range(len(sent2)):
        #    w2 = sent2[j]
        #    sim = 0
        #    if w2 in density_tfidf.index:
        #        #index of word with max cosine similarity from sent1
        #        max_j = np.argmax(sim_mat.transpose()[j])

        #        #multiply the tfidf value of current word of sent2 with cosine similarity of the most similar from sent1
        #        sim = sim_mat.transpose()[j][max_j]*density_tfidf['s2_tfidf'][w2]
        #    else:
        #        sim = 0
        #    W_t_s+=sim
        
        
        #k = k + 1
        #print('k: ', k)

        return W_s_t#, sim_mat, max_cosine_vec, sent1_tf_vec, sent1, sent2   #min(W_s_t, W_t_s)
    except:
        #print('sent_1: ', sent_1)
        sent_1_emb = get_sentence_embedding(sent1, word_model)
        #print('sent_2: ', sent_2)
        sent_2_emb = get_sentence_embedding(sent2, word_model)
        
        #k = k + 1
        #print('k: ', k)
        
        return sent_1_emb.dot(sent_2_emb.T)#, 0, 0, 0, 0, 0#cosine_sim(sent_1_emb, sent_2_emb)

In [29]:
#Convert ICD Indices to hot vectors of length: NUM_ICD_CODE
def generate_binary(icd_index_list, total_icd = NUM_ICD_CODES):
    binary_icd = np.zeros(total_icd)
    for x in icd_index_list:
        binary_icd[x] = 1
        
    return binary_icd

In [30]:
#Calculate Specificity

def calc_specificity(pred_vec, act_vec):
    m = len(pred_vec)
    
    specificity = []
    
    for i in range(m):
        pred, act = pred_vec[i], act_vec[i]
        dict_XY = {'pred':pred, 'act':act}
        res = pd.DataFrame(dict_XY, columns=['pred', 'act'])
        res['Sum'] = res['pred'] + res['act']
        
        pred_p = res[res.pred == 1]
        pred_n = res[res.act == 1]
        
        tp = len(res[res.Sum == 2])
        tn = len(res[res.Sum == 0]) 
        
        fp = len(pred_p[pred_p.act == 0])
        fn = len(pred_n[pred_n.act == 1])
        
        #Calculate specificity for this example:
        spec = tn/(tn + fp)
        
        specificity.append(spec)
    
    return sum(specificity) / float(m)
        

In [46]:
#Predict ICD 9 Codes

#construct the Term Frequency Vector using just the icd corpus
c_vectorizer, freq_vec = get_tf(df_icd_cleaned['Long Description'])

start_time = time.time()

#Calculate the Similarity of each document with each icd text
X = DD_Data['clean_text']#[0]
Y = list(DD_Data['ICD9_CODE'])
m = len(Y)

sim = []

for i in range(1000,m):
    text = X[i]
    #Modified#Calculate cosine similarity of sentence with icd sentences
    sim.append(np.array([RWMS_similarity(icd_text, text, freq_vec) for icd_text in icd_sentences]))#cosine_sim1_matrix(sent_emb)
    
    #reset debugging counter
    #global k
    #k = 0
    
    if (i+1)%10 == 0:
        print(str(i+1) + '/' + str(m) + ' records done.')
    
    if (i+1)%100 == 0:
        print('\nTime elapsed: ', (time.time() - start_time)/60.0, ' minutes')

    if (i+1)%1000 == 0:
        sim_df = pd.DataFrame(sim, dtype = float)
        sim_df.to_csv('Similarity_Data/Optimized_NP_RWMS_similarity_data_' + str(i+1) + '_records.csv')
        #sim_df.to_csv('Optimized_NP_RWMS_similarity_data_' + (i+1) + '_records.csv')

time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')

#Save the similarity scores in an excel file
print('Saving Similarity scores in file......')
sim_df = pd.DataFrame(sim, dtype = float)
sim_df.to_csv('Similarity_Data/Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')
#sim_df.to_csv('Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')
print('Similarity Scores Saved!')

1010/22722 records done.
1020/22722 records done.
1030/22722 records done.
1040/22722 records done.
1050/22722 records done.
1060/22722 records done.
1070/22722 records done.
1080/22722 records done.
1090/22722 records done.
1100/22722 records done.

Time elapsed:  64.77565616369247  minutes
1110/22722 records done.
1120/22722 records done.
1130/22722 records done.
1140/22722 records done.
1150/22722 records done.
1160/22722 records done.
1170/22722 records done.
1180/22722 records done.
1190/22722 records done.
1200/22722 records done.

Time elapsed:  126.7612806638082  minutes
1210/22722 records done.
1220/22722 records done.
1230/22722 records done.
1240/22722 records done.
1250/22722 records done.
1260/22722 records done.
1270/22722 records done.
1280/22722 records done.
1290/22722 records done.
1300/22722 records done.

Time elapsed:  196.83429719607037  minutes
1310/22722 records done.
1320/22722 records done.
1330/22722 records done.
1340/22722 records done.
1350/22722 records d

3810/22722 records done.
3820/22722 records done.
3830/22722 records done.
3840/22722 records done.
3850/22722 records done.
3860/22722 records done.
3870/22722 records done.
3880/22722 records done.
3890/22722 records done.
3900/22722 records done.

Time elapsed:  1918.9585484941801  minutes
3910/22722 records done.
3920/22722 records done.
3930/22722 records done.
3940/22722 records done.
3950/22722 records done.
3960/22722 records done.
3970/22722 records done.
3980/22722 records done.
3990/22722 records done.
4000/22722 records done.

Time elapsed:  1981.952932047844  minutes
4010/22722 records done.
4020/22722 records done.
4030/22722 records done.
4040/22722 records done.
4050/22722 records done.
4060/22722 records done.
4070/22722 records done.
4080/22722 records done.
4090/22722 records done.
4100/22722 records done.

Time elapsed:  2045.8291783809661  minutes
4110/22722 records done.
4120/22722 records done.
4130/22722 records done.
4140/22722 records done.
4150/22722 records 

6610/22722 records done.
6620/22722 records done.
6630/22722 records done.
6640/22722 records done.
6650/22722 records done.
6660/22722 records done.
6670/22722 records done.
6680/22722 records done.
6690/22722 records done.
6700/22722 records done.

Time elapsed:  3700.2030342976254  minutes
6710/22722 records done.
6720/22722 records done.
6730/22722 records done.
6740/22722 records done.
6750/22722 records done.
6760/22722 records done.
6770/22722 records done.
6780/22722 records done.
6790/22722 records done.
6800/22722 records done.

Time elapsed:  3772.173684966564  minutes
6810/22722 records done.
6820/22722 records done.
6830/22722 records done.
6840/22722 records done.
6850/22722 records done.
6860/22722 records done.
6870/22722 records done.
6880/22722 records done.
6890/22722 records done.
6900/22722 records done.

Time elapsed:  3837.0083461443583  minutes
6910/22722 records done.
6920/22722 records done.
6930/22722 records done.
6940/22722 records done.
6950/22722 records 

9410/22722 records done.
9420/22722 records done.
9430/22722 records done.
9440/22722 records done.
9450/22722 records done.
9460/22722 records done.
9470/22722 records done.
9480/22722 records done.
9490/22722 records done.
9500/22722 records done.

Time elapsed:  5508.446252314249  minutes
9510/22722 records done.
9520/22722 records done.
9530/22722 records done.
9540/22722 records done.
9550/22722 records done.
9560/22722 records done.
9570/22722 records done.
9580/22722 records done.
9590/22722 records done.
9600/22722 records done.

Time elapsed:  5571.952359116078  minutes
9610/22722 records done.
9620/22722 records done.
9630/22722 records done.
9640/22722 records done.
9650/22722 records done.
9660/22722 records done.
9670/22722 records done.
9680/22722 records done.
9690/22722 records done.
9700/22722 records done.

Time elapsed:  5634.452756480376  minutes
9710/22722 records done.
9720/22722 records done.
9730/22722 records done.
9740/22722 records done.
9750/22722 records do

12140/22722 records done.
12150/22722 records done.
12160/22722 records done.
12170/22722 records done.
12180/22722 records done.
12190/22722 records done.
12200/22722 records done.

Time elapsed:  7242.295690731207  minutes
12210/22722 records done.
12220/22722 records done.
12230/22722 records done.
12240/22722 records done.
12250/22722 records done.
12260/22722 records done.
12270/22722 records done.
12280/22722 records done.
12290/22722 records done.
12300/22722 records done.

Time elapsed:  7309.641447714965  minutes
12310/22722 records done.
12320/22722 records done.
12330/22722 records done.
12340/22722 records done.
12350/22722 records done.
12360/22722 records done.
12370/22722 records done.
12380/22722 records done.
12390/22722 records done.
12400/22722 records done.

Time elapsed:  7375.975706326962  minutes
12410/22722 records done.
12420/22722 records done.
12430/22722 records done.
12440/22722 records done.
12450/22722 records done.
12460/22722 records done.
12470/22722 r

14850/22722 records done.
14860/22722 records done.
14870/22722 records done.
14880/22722 records done.
14890/22722 records done.
14900/22722 records done.

Time elapsed:  9000.433986266455  minutes
14910/22722 records done.
14920/22722 records done.
14930/22722 records done.
14940/22722 records done.
14950/22722 records done.
14960/22722 records done.
14970/22722 records done.
14980/22722 records done.
14990/22722 records done.
15000/22722 records done.

Time elapsed:  9069.268155026435  minutes
15010/22722 records done.
15020/22722 records done.
15030/22722 records done.
15040/22722 records done.
15050/22722 records done.
15060/22722 records done.
15070/22722 records done.
15080/22722 records done.
15090/22722 records done.
15100/22722 records done.

Time elapsed:  9131.16069656213  minutes
15110/22722 records done.
15120/22722 records done.
15130/22722 records done.
15140/22722 records done.
15150/22722 records done.
15160/22722 records done.
15170/22722 records done.
15180/22722 re

17560/22722 records done.
17570/22722 records done.
17580/22722 records done.
17590/22722 records done.
17600/22722 records done.

Time elapsed:  10833.559555013975  minutes
17610/22722 records done.
17620/22722 records done.
17630/22722 records done.
17640/22722 records done.
17650/22722 records done.
17660/22722 records done.
17670/22722 records done.
17680/22722 records done.
17690/22722 records done.
17700/22722 records done.

Time elapsed:  10911.056475762527  minutes
17710/22722 records done.
17720/22722 records done.
17730/22722 records done.
17740/22722 records done.
17750/22722 records done.
17760/22722 records done.
17770/22722 records done.
17780/22722 records done.
17790/22722 records done.
17800/22722 records done.

Time elapsed:  10988.288394212723  minutes
17810/22722 records done.
17820/22722 records done.
17830/22722 records done.
17840/22722 records done.
17850/22722 records done.
17860/22722 records done.
17870/22722 records done.
17880/22722 records done.
17890/2272

20260/22722 records done.
20270/22722 records done.
20280/22722 records done.
20290/22722 records done.
20300/22722 records done.

Time elapsed:  12693.975648562113  minutes
20310/22722 records done.
20320/22722 records done.
20330/22722 records done.
20340/22722 records done.
20350/22722 records done.
20360/22722 records done.
20370/22722 records done.
20380/22722 records done.
20390/22722 records done.
20400/22722 records done.

Time elapsed:  12762.71708583037  minutes
20410/22722 records done.
20420/22722 records done.
20430/22722 records done.
20440/22722 records done.
20450/22722 records done.
20460/22722 records done.
20470/22722 records done.
20480/22722 records done.
20490/22722 records done.
20500/22722 records done.

Time elapsed:  12827.2392081817  minutes
20510/22722 records done.
20520/22722 records done.
20530/22722 records done.
20540/22722 records done.
20550/22722 records done.
20560/22722 records done.
20570/22722 records done.
20580/22722 records done.
20590/22722 r

In [47]:
#Load Similarity Data
sim = pd.read_csv('Similarity_Data/Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')
#sim = pd.read_csv('Optimized_NP_RWMS_similarity_data_' + str(m) + '_records.csv')

#Drop first column
sim = sim.drop(['Unnamed: 0'], axis = 1)

#Convert DataFrame to List
sim = np.array(sim.values.tolist())

In [61]:
sim.shape

(22722, 2833)

In [49]:
#Save Predictions and Corresponding Scores per threshold

#Initialise Similarity Result Dataframe
results_sim = pd.DataFrame(columns = ['Actual Code', 
                                  'Predicted Codes',
                                  'Similarity Codes'])


threshold_list = [0.6, 0.7, 0.8, 0.9]

#Initialize Excel
#writer = pd.ExcelWriter('Similarity_Data/Code and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records.xlsx', engine='xlsxwriter')

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)

    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])

    data_input_format = []

    i = 0
    for i in range(m):
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_codes = [str(y) for y in Y[i][2:-2].split("', '") if y in icd_codes]#To get rid of '' and ,
        act_codes_string = ', '.join(act_codes)

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh


        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred_indices = np.nonzero(sim_post_threshold)[0].tolist()

        #Get codes from indices
        pred_codes = [str(dict_index_code[p]) for p in pred_indices]
        #Convert to Comma Separated String
        pred_codes_string = ', '.join(pred_codes)
        #print([pred_codes_string])

        #Corresponding Similarity Scores of List of Indices
        sim_scores = [str(sim[i][j]) for j in pred_indices]
        #Convert to Comma Separated String
        sim_scores_string = ', '.join(sim_scores)
        #print([sim_scores_string])


        data_input = [act_codes_string, pred_codes_string, sim_scores_string]

        #Append
        data_input_format.append(data_input)

    df = pd.DataFrame(data_input_format, columns = ['Actual Codes', 'Predicted Codes', 'Similarity Scores'])
    df.to_csv('Similarity_Data/Code and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')
    #df.to_csv('NP - Predicted Codes and Similarity Data - Unsupervised ICD - ' + str(m) + ' Records_' + str(thresh) + '_threshold.csv')


Commenced for Similarity Threshold: 0.6

Commenced for Similarity Threshold: 0.7

Commenced for Similarity Threshold: 0.8

Commenced for Similarity Threshold: 0.9


In [50]:
#Calculate Metrics

#Initialize Results Dataframe which will store the relevant metrics for each threshold
results = pd.DataFrame(columns = ['Number of Observations', 
                                  'Number of ICD Codes',
                                  'Threshold',
                                  'Precision', 
                                  'Recall', 
                                  'F1-Score', 
                                  'Specificity'])

#Compute Accuracy metrics based on thresholding        
#threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
threshold_list = [0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.4]
#threshold_list = [1.0, 5.0 10.0, 15.0, 20.0]

start_time = time.time()

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)
    
    #X = DD_Data['clean_text'][0]
    #Y = list(DD_Data['ICD9_CODE'])
    
    predicted_codes_index = []
    actual_codes_index = []

    #m = 1#len(Y)

    for i in range(m):
        #text = X#[i]
        #print(i)
        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_icd_indices = [dict_code_index[y] for y in Y[i][2:-2].split("', '") if y in icd_codes] #To get rid of '' and ,

        #Initialize Prediction list. This will contain indices of similar icd codes
        pred = []

        #Apply thresholding to obtain indices of similar embeddings
        #print(sim)
        threshold_filter = sim[i] > thresh

        
        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim[i]*threshold_filter

        #Add indices of non zero elements to pred
        pred.extend(np.nonzero(sim_post_threshold)[0].tolist())

        #Consider the unique indices
        pred = set(pred)

        #Convert list of indices to hot vector
        pred_icd_vector = generate_binary(pred)
        act_icd_vector = generate_binary(act_icd_indices)

        #Append
        predicted_codes_index.append(pred_icd_vector)
        actual_codes_index.append(act_icd_vector)


        #if i%100 == 0 and i != 0:
        #    print(i, ' sentences done.')

    #Calculate Precision, Recall, F1 Score for this threshold
    precision, recall, f1_score, support = precision_recall_fscore_support(actual_codes_index, predicted_codes_index, average = 'micro')
    specificity = calc_specificity(predicted_codes_index, actual_codes_index)

    #Add to results Dataframe:
    results = results.append({'Number of Observations': m, 
                    'Number of ICD Codes': NUM_ICD_CODES, 
                    'Threshold': thresh,
                    'Precision': precision, 
                    'Recall': recall, 
                    'F1-Score': f1_score, 
                    'Specificity': specificity}, ignore_index=True)

    print('\nPrecision: ', precision, ' Recall: ', recall, ' F1-Score: ', f1_score, ' Specificity:', specificity)


time_elapsed = time.time() - start_time
print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')    
    
#Print Results
RESULT_FILENAME = 'Results/Experiment_Results_RWMS_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
#RESULT_FILENAME = 'Experiment_Results_NP_RWMS_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
results.to_excel(RESULT_FILENAME) 
print('Results Saved!')
results


Commenced for Similarity Threshold: 0.6

Precision:  0.0054585525866192855  Recall:  0.5854352429669575  F1-Score:  0.010816255252101815  Specificity: 0.5859026090913833

Commenced for Similarity Threshold: 0.7

Precision:  0.007418156773625595  Recall:  0.4359220609095037  F1-Score:  0.014588066049175487  Specificity: 0.7735367992216762

Commenced for Similarity Threshold: 0.8

Precision:  0.010333709043101862  Recall:  0.3152442528158289  F1-Score:  0.020011442835435652  Specificity: 0.8827679516107128

Commenced for Similarity Threshold: 0.9

Precision:  0.013691919914978373  Recall:  0.21059547297487352  F1-Score:  0.02571215718616011  Specificity: 0.9410908961608111

Time elapsed:  6.594137016932169  minutes
Results Saved!


,Number of Observations,Number of ICD Codes,Threshold,Precision,Recall,F1-Score,Specificity
0,22722.0,2833.0,0.6,0.005459,0.585435,0.010816,0.585903
1,22722.0,2833.0,0.7,0.007418,0.435922,0.014588,0.773537
2,22722.0,2833.0,0.8,0.010334,0.315244,0.020011,0.882768
3,22722.0,2833.0,0.9,0.013692,0.210595,0.025712,0.941091
